# Praktikum Pengantar Pembelajaran Mesin
---
## Bab 8. Metode Evaluasi Algoritma Klasifikasi
Anggota :
1. Riza Setiawan Soetedjo / 195150207111062
2. Dominicus Christian Bagus Susanto / 195150200111086
3. Muhammad Haris Khoirul Anam / 195150207111063


##Dataset In-vehicle Coupon Recommendation Dataset

###1. Pengambilan Data

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv

--2021-06-11 09:04:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2163456 (2.1M) [application/x-httpd-php]
Saving to: ‘in-vehicle-coupon-recommendation.csv’

in-vehicle-coupon-r 100%[===================>]   2.06M  4.55MB/s    in 0.5s    

2021-06-11 09:04:18 (4.55 MB/s) - ‘in-vehicle-coupon-recommendation.csv’ saved [2163456/2163456]



In [ ]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('in-vehicle-coupon-recommendation.csv')

In [ ]:
dataset

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,has_children,education,occupation,income,car,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,Home,Partner,Rainy,55,6PM,Carry out & Take away,1d,Male,26,Single,0,Bachelors degree,Sales & Related,$75000 - $87499,NaN,never,never,1~3,4~8,1~3,1,0,0,1,0,1
12680,Work,Alone,Rainy,55,7AM,Carry out & Take away,1d,Male,26,Single,0,Bachelors degree,Sales & Related,$75000 - $87499,NaN,never,never,1~3,4~8,1~3,1,0,0,0,1,1
12681,Work,Alone,Snowy,30,7AM,Coffee House,1d,Male,26,Single,0,Bachelors degree,Sales & Related,$75000 - $87499,NaN,never,never,1~3,4~8,1~3,1,0,0,1,0,0
12682,Work,Alone,Snowy,30,7AM,Bar,1d,Male,26,Single,0,Bachelors degree,Sales & Related,$75000 - $87499,NaN,never,never,1~3,4~8,1~3,1,1,1,0,1,0


###2. Pemrosesan Awal Data

In [ ]:
print("Banyak baris yang NaN di tiap kolom")
dataset.isnull().sum()

Banyak baris yang NaN di tiap kolom


destination                 0
passanger                   0
weather                     0
temperature                 0
time                        0
coupon                      0
expiration                  0
gender                      0
age                         0
maritalStatus               0
has_children                0
education                   0
occupation                  0
income                      0
car                     12576
Bar                       107
CoffeeHouse               217
CarryAway                 151
RestaurantLessThan20      130
Restaurant20To50          189
toCoupon_GEQ5min            0
toCoupon_GEQ15min           0
toCoupon_GEQ25min           0
direction_same              0
direction_opp               0
Y                           0
dtype: int64

In [ ]:
#Melakukan imputasi dengan cara membuat nilai NaN sebagai kategori baru yaitu "Unknown"
def impute_nan_create_category(DataFrame,ColName):
     DataFrame[ColName] = np.where(DataFrame[ColName].isnull(),"Unknown",DataFrame[ColName])

for Columns in dataset.columns[:-1]:
    impute_nan_create_category(dataset,Columns)

In [ ]:
#Melakukan Integer Encoding dimana tiap nilai unik pada tiap kolom memiliki representasi angka
from pprint import pprint
list_fitur = {}
for nama_fitur in dataset.columns[:-1]:
  list_fitur[nama_fitur] = {}
  list_kategori = np.array(dataset[nama_fitur].unique())
  temp = {}
  for i in range(list_kategori.shape[0]):
    temp[i] = list_kategori[i]
    dataset[nama_fitur] = np.where(dataset[nama_fitur]==list_kategori[i],i,dataset[nama_fitur])
  list_fitur[nama_fitur] = temp
pprint(list_fitur)

{'Bar': {0: 'never', 1: 'less1', 2: '1~3', 3: 'gt8', 4: 'Unknown', 5: '4~8'},
 'CarryAway': {0: 'Unknown',
               1: '4~8',
               2: '1~3',
               3: 'gt8',
               4: 'less1',
               5: 'never'},
 'CoffeeHouse': {0: 'never',
                 1: 'less1',
                 2: '4~8',
                 3: '1~3',
                 4: 'gt8',
                 5: 'Unknown'},
 'Restaurant20To50': {0: '1~3',
                      1: 'less1',
                      2: 'never',
                      3: 'gt8',
                      4: '4~8',
                      5: 'Unknown'},
 'RestaurantLessThan20': {0: '4~8',
                          1: '1~3',
                          2: 'less1',
                          3: 'gt8',
                          4: 'Unknown',
                          5: 'never'},
 'age': {0: '21',
         1: '46',
         2: '26',
         3: '31',
         4: '41',
         5: '50plus',
         6: '36',
         7: 'below21'},
 'car': {0: 

In [ ]:
#Menghapus tuple yang duplikat
print('Banyak tuple sebelum duplikasi=', dataset.shape[0])
dataset = dataset.drop_duplicates()
print('Banyak tuple setelah duplikasi=', dataset.shape[0])

Banyak tuple sebelum duplikasi= 12684
Banyak tuple setelah duplikasi= 12610


In [ ]:
#Memilih fitur yang dihilangkan sesuai dari variansinya
from sklearn.feature_selection import VarianceThreshold
X = dataset.values[:,:-1]#fitur
nama_fitur = dataset.columns[:-1]
selector = VarianceThreshold(threshold=0.1)
selector.fit(X)
deleted_fitur = []
for i in range(len(nama_fitur)):
  if selector.get_support()[i] == False:
    deleted_fitur.append(nama_fitur[i])
print("Fitur yang dihapus\n", deleted_fitur)

Fitur yang dihapus
 ['car', 'toCoupon_GEQ5min']


In [ ]:
#Menghapus kolom yang tidak diperlukan
for x in deleted_fitur:
  del dataset[x]
dataset.sample(5)

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,has_children,education,occupation,income,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
2241,1,0,0,0,2,4,0,0,2,2,1,2,7,2,0,1,3,0,2,1,0,0,0,0
8705,0,3,2,2,0,1,1,1,3,2,1,4,19,3,2,2,4,0,1,1,0,0,0,1
2021,1,0,0,0,2,4,0,0,5,2,0,2,15,6,1,0,3,0,0,1,0,0,0,1
9246,2,0,1,0,3,3,0,1,5,2,0,0,13,1,2,0,2,1,1,1,1,0,0,1
10660,2,0,0,1,3,1,0,1,0,0,1,0,2,8,1,3,1,0,0,0,0,1,1,0


###3. Pemilihan Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

In [ ]:
# Model yang dipakai
models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', MultinomialNB()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('Random Forest', RandomForestClassifier()))

# Menyimpan model dan hasilnya
results = []
names = []

In [ ]:
#Melakukan pemisahan menjadi data latih, label latih, data validasi, dan label validasi
array = dataset.values
X = array[:,:-1]#fitur
y = array[:,-1]#kelas
y=y.astype('int')

X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.2, 
                                                                random_state=101, shuffle=True)

In [ ]:
#Melihat perkiraan akurasi untuk tiap model yang digunakan
for name, model in models:
  cv_results = cross_val_score(model, X_train, Y_train, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  print('%s: %0.2f akurasi dengan standar deviasi %f' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.63 akurasi dengan standar deviasi 0.010854
NB: 0.62 akurasi dengan standar deviasi 0.005104
Decision Tree: 0.67 akurasi dengan standar deviasi 0.007493
SVM: 0.67 akurasi dengan standar deviasi 0.008875
Random Forest: 0.74 akurasi dengan standar deviasi 0.004958


In [ ]:
#KNN
parameters = {'n_neighbors':[3,5,10],'weights':('uniform','distance'),'algorithm':('ball_tree','kd_tree','brute')}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters, return_train_score=True)
clf.fit(X_train,Y_train)

pd.concat([pd.DataFrame(clf.cv_results_["params"]),
           pd.DataFrame(clf.cv_results_["mean_train_score"],columns=["Training accuracy"]),
           pd.DataFrame(clf.cv_results_["mean_test_score"], columns=["Testing accuracy"]),
           pd.DataFrame(clf.cv_results_["rank_test_score"],columns=["Ranking"])],
          axis=1).sort_values(by=['Ranking'])

,algorithm,n_neighbors,weights,Training accuracy,Testing accuracy,Ranking
11,kd_tree,10,distance,0.998736,0.648197,1
5,ball_tree,10,distance,0.998736,0.647305,2
17,brute,10,distance,0.998736,0.645818,3
10,kd_tree,10,uniform,0.722690,0.642843,4
4,ball_tree,10,uniform,0.723384,0.642049,5
16,brute,10,uniform,0.723310,0.641753,6
15,brute,5,distance,0.998736,0.637590,7
9,kd_tree,5,distance,0.998736,0.636896,8
3,ball_tree,5,distance,0.998736,0.636500,9
14,brute,5,uniform,0.760384,0.633624,10


In [ ]:
#Naive Bayes
parameters = {'alpha':[0.1,1.0,2.0,3.0,1.5],'fit_prior':(True, False)}
naive_bayes = MultinomialNB()
clf = GridSearchCV(naive_bayes, parameters, return_train_score=True)
clf.fit(X_train,Y_train)

pd.concat([pd.DataFrame(clf.cv_results_["params"]),
           pd.DataFrame(clf.cv_results_["mean_train_score"],columns=["Training accuracy"]),
           pd.DataFrame(clf.cv_results_["mean_test_score"], columns=["Testing accuracy"]),
           pd.DataFrame(clf.cv_results_["rank_test_score"],columns=["Ranking"])],
          axis=1).sort_values(by=['Ranking'])

,alpha,fit_prior,Training accuracy,Testing accuracy,Ranking
4,2.0,True,0.624579,0.621035,1
8,1.5,True,0.624628,0.621035,1
0,0.1,True,0.624703,0.620936,3
2,1.0,True,0.624653,0.620936,3
6,3.0,True,0.624604,0.620837,5
3,1.0,False,0.610701,0.609934,6
5,2.0,False,0.610775,0.609934,6
7,3.0,False,0.610800,0.609934,6
9,1.5,False,0.610775,0.609934,6
1,0.1,False,0.610701,0.609835,10


In [ ]:
#Decision Tree	
parameters = {'splitter':('best', 'random'),'criterion':('gini', 'entropy'),'random_state':[1,51,101]}
d_tree = DecisionTreeClassifier()
clf = GridSearchCV(d_tree, parameters, return_train_score=True)
clf.fit(X_train,Y_train)
pd.concat([pd.DataFrame(clf.cv_results_["params"]),
           pd.DataFrame(clf.cv_results_["mean_train_score"],columns=["Training accuracy"]),
           pd.DataFrame(clf.cv_results_["mean_test_score"], columns=["Testing accuracy"]),
           pd.DataFrame(clf.cv_results_["rank_test_score"],columns=["Ranking"])],
          axis=1).sort_values(by=['Ranking'])

,criterion,random_state,splitter,Training accuracy,Testing accuracy,Ranking
10,entropy,101,best,0.998736,0.678728,1
8,entropy,51,best,0.998736,0.678628,2
6,entropy,1,best,0.998736,0.677935,3
0,gini,1,best,0.998736,0.667921,4
4,gini,101,best,0.998736,0.665543,5
2,gini,51,best,0.998736,0.665345,6
9,entropy,51,random,0.998736,0.662966,7
7,entropy,1,random,0.998736,0.661183,8
3,gini,51,random,0.998736,0.657216,9
1,gini,1,random,0.998736,0.655137,10


In [ ]:
#SVM
parameters = {'kernel':('linear', 'rbf', 'sigmoid'),'gamma':('scale', 'auto'),'random_state':[1,51,101]}
svm = SVC()
clf = GridSearchCV(svm, parameters, return_train_score=True)
clf.fit(X_train,Y_train)
pd.concat([pd.DataFrame(clf.cv_results_["params"]),
           pd.DataFrame(clf.cv_results_["mean_train_score"],columns=["Training accuracy"]),
           pd.DataFrame(clf.cv_results_["mean_test_score"], columns=["Testing accuracy"]),
           pd.DataFrame(clf.cv_results_["rank_test_score"],columns=["Ranking"])],
          axis=1).sort_values(by=['Ranking'])

,gamma,kernel,random_state,Training accuracy,Testing accuracy,Ranking
14,auto,rbf,101,0.784149,0.674564,1
13,auto,rbf,51,0.784149,0.674564,1
12,auto,rbf,1,0.784149,0.674564,1
3,scale,rbf,1,0.678058,0.656027,4
4,scale,rbf,51,0.678058,0.656027,4
5,scale,rbf,101,0.678058,0.656027,4
0,scale,linear,1,0.638159,0.635309,7
1,scale,linear,51,0.638159,0.635309,7
2,scale,linear,101,0.638159,0.635309,7
9,auto,linear,1,0.638159,0.635309,7


In [ ]:
#Random Forest	
parameters = {'n_estimators':[50,100],'criterion':('gini', 'entropy'),'random_state':[1,51,101]}
rand_forest = RandomForestClassifier()
clf = GridSearchCV(rand_forest, parameters, return_train_score=True)
clf.fit(X_train,Y_train)
pd.concat([pd.DataFrame(clf.cv_results_["params"]),
           pd.DataFrame(clf.cv_results_["mean_train_score"],columns=["Training accuracy"]),
           pd.DataFrame(clf.cv_results_["mean_test_score"], columns=["Testing accuracy"]),
           pd.DataFrame(clf.cv_results_["rank_test_score"],columns=["Ranking"])],
          axis=1).sort_values(by=['Ranking'])

,criterion,n_estimators,random_state,Training accuracy,Testing accuracy,Ranking
5,gini,100,101,0.998736,0.744945,1
3,gini,100,1,0.998736,0.744549,2
9,entropy,100,1,0.998736,0.744252,3
10,entropy,100,51,0.998736,0.743062,4
4,gini,100,51,0.998736,0.741178,5
2,gini,50,101,0.998538,0.740187,6
11,entropy,100,101,0.998711,0.739890,7
7,entropy,50,51,0.998711,0.739197,8
6,entropy,50,1,0.998587,0.737909,9
0,gini,50,1,0.998662,0.737610,10


###4. Pelatihan Model

In [ ]:
modelKNN = KNeighborsClassifier(algorithm='kd_tree',n_neighbors=10,weights='distance')
modelKNN.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [ ]:
modelNB = MultinomialNB(alpha=2.0, fit_prior=True)
modelNB.fit(X_train,Y_train)

MultinomialNB(alpha=2.0, class_prior=None, fit_prior=True)

In [ ]:
modelDT = DecisionTreeClassifier(criterion='entropy',splitter='best',random_state=101)
modelDT.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=101, splitter='best')

In [ ]:
modelSVM = SVC(gamma='auto',kernel='rbf',random_state=101)
modelSVM.fit(X_train,Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=101, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
modelRF = RandomForestClassifier(criterion='gini',n_estimators=100, random_state=101)
modelRF.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=101,
                       verbose=0, warm_start=False)

###5. Evaluasi Model

In [ ]:
#Fungsi untuk melakukan prediksi sesuai model
def prediksi(model,X_train,Y_train,X_validation):
  predictions = model.predict(X_validation)
  return predictions

In [ ]:
prediction = prediksi(modelKNN, X_train, Y_train,  X_validation)
print("Hasil K-Nearest Neighbour:")
print(classification_report(Y_validation, prediction),"\n")

Hasil K-Nearest Neighbour:
              precision    recall  f1-score   support

           0       0.60      0.52      0.56      1074
           1       0.68      0.74      0.71      1448

    accuracy                           0.65      2522
   macro avg       0.64      0.63      0.63      2522
weighted avg       0.64      0.65      0.64      2522
 



In [ ]:
prediction = prediksi(modelNB, X_train, Y_train,  X_validation)
print("Hasil Naive Bayes:")
print(classification_report(Y_validation, prediction),"\n")

Hasil Naive Bayes:
              precision    recall  f1-score   support

           0       0.56      0.45      0.50      1074
           1       0.64      0.74      0.69      1448

    accuracy                           0.61      2522
   macro avg       0.60      0.59      0.59      2522
weighted avg       0.61      0.61      0.61      2522
 



In [ ]:
prediction = prediksi(modelDT, X_train, Y_train,  X_validation)
print("Hasil Decision Tree:")
print(classification_report(Y_validation, prediction),"\n")

Hasil Decision Tree:
              precision    recall  f1-score   support

           0       0.63      0.63      0.63      1074
           1       0.73      0.72      0.72      1448

    accuracy                           0.68      2522
   macro avg       0.68      0.68      0.68      2522
weighted avg       0.68      0.68      0.68      2522
 



In [ ]:
prediction = prediksi(modelSVM, X_train, Y_train,  X_validation)
print("Hasil Support Vector Machine:")
print(classification_report(Y_validation, prediction),"\n")

Hasil Support Vector Machine:
              precision    recall  f1-score   support

           0       0.64      0.54      0.59      1074
           1       0.69      0.77      0.73      1448

    accuracy                           0.67      2522
   macro avg       0.67      0.66      0.66      2522
weighted avg       0.67      0.67      0.67      2522
 



In [ ]:
prediction = prediksi(modelRF, X_train, Y_train,  X_validation)
print("Hasil Random Forest:")
print(classification_report(Y_validation, prediction),"\n")

Hasil Random Forest:
              precision    recall  f1-score   support

           0       0.73      0.65      0.69      1074
           1       0.76      0.83      0.79      1448

    accuracy                           0.75      2522
   macro avg       0.75      0.74      0.74      2522
weighted avg       0.75      0.75      0.75      2522
 

